# AI Support Copilot - Complete Demo

This notebook demonstrates the **entire AI Support Copilot system** end-to-end.

## What We'll Cover

1. **Setup & Verification** - Check environment
2. **MCP Server Overview** - Understanding tools
3. **Test Individual Tools** - See MCP in action
4. **Process a Ticket** - Complete multi-agent flow
5. **Explore Traces** - Observability deep dive
6. **Test All Agents Individually** - Understand each component

---

## Prerequisites

**Before running this notebook:**
1. MCP server must be running in a terminal:
   ```bash
   python -m support_mcp_server.server
   ```
2. Set your OpenAI API key:
   ```bash
   export OPENAI_API_KEY="your-key"
   ```

## 1. Setup & Imports

In [6]:
import pathlib, os
cwd = pathlib.Path().resolve()
print(f"Current working directory: {cwd}")
if cwd.name == "notebooks_ref":
    os.chdir(cwd.parent)

Current working directory: /Users/shivam13juna/Documents/scaler/ExtraClasses/rev_iitr/hands_on/ref


In [7]:
# Standard imports
import json
import asyncio
import os
from pathlib import Path

# Rich for beautiful output
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.syntax import Syntax
from rich.markdown import Markdown

# Our project imports
from support_copilot_host.models import Ticket, Trace
from support_copilot_host.orchestrator import run_ticket_flow
from support_copilot_host.agents import TriageAgent, ResearchAgent, ActionAgent, SupervisorAgent
from support_copilot_host.mcp_client import SupportMCPClient
from support_copilot_host.observability import render_trace_to_stdout, print_trace_summary

console = Console()
console.print("[bold green]✓ All imports successful![/bold green]")

✓ All imports successful!

In [8]:
# Verify environment
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    console.print(f"[green]✓ OpenAI API key found ({api_key[:10]}...)[/green]")
else:
    console.print("[red]✗ OPENAI_API_KEY not set![/red]")
    console.print("Set it with: export OPENAI_API_KEY='your-key'")

✓ OpenAI API key found (sk-proj-Ln...)

In [ ]:


# Load sample tickets
with open("data/tickets/samples.json") as f:
    sample_tickets = json.load(f)

console.print(f"[cyan]Loaded {len(sample_tickets)} sample tickets[/cyan]")
for ticket in sample_tickets:
    console.print(f"  • {ticket['id']}: {ticket['description'][:60]}...")

Loaded 4 sample tickets

• TICKET-001: Customer cannot export their main workspace dashboard to CSV...

• TICKET-002: Several SSO users in the EU are failing to log in via SAML. ...

• TICKET-003: User reports they are an editor on a workspace but cannot ed...

• TICKET-004: Customer's export API integration started failing with HTTP ...

In [5]:
sample_tickets[0]

{'id': 'TICKET-001',
 'description': "Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout' after ~30 seconds.",
 'screenshot_path': 'data/tickets/screenshots/export_timeout.png',
 'log_snippet': '2025-11-01T09:21:10Z ERR_EXPORT_TIMEOUT workspace_id=ws_1234 service=export_service'}

## 2. MCP Server & Tools Overview

The MCP server exposes 3 tools:
- `support_docs.search` - Search runbooks
- `incidents.search` - Find related incidents
- `status.check` - Check service health

In [4]:
# Connect to MCP server
mcp_client = SupportMCPClient()

async def connect_mcp():
    await mcp_client.connect()
    console.print("[green]✓ Connected to MCP server![/green]")

await connect_mcp()

✓ Connected to MCP server!

### 2.1 Test Tool: support_docs.search

In [5]:
# Search for export-related documentation
docs_results = await mcp_client.call_support_docs_search(
    query="export csv timeout",
    max_results=3
)

console.print("\n[bold cyan]Support Docs Search Results:[/bold cyan]")
for doc in docs_results:
    console.print(Panel(
        f"[yellow]Score: {doc['score']}[/yellow]\n\n{doc['snippet']}",
        title=f"[bold]{doc['title']}[/bold]",
        subtitle=doc['path'],
        border_style="cyan"
    ))

Support Docs Search Results:

╭───────────────────────────────────────────── Export to CSV Errors ──────────────────────────────────────────────╮
│ Score: 15                                                                                                       │
│                                                                                                                 │
│ # Export to CSV Errors                                                                                          │
│                                                                                                                 │
│ When users attempt to export reports to CSV and see errors like                                                 │
│ **"Export failed: timeout"** or **"Export failed: invalid workspace"**, follow                                  │
│ these steps:                                                                                                    │
│                                                                                                                 │
│ 1. Confirm the user has the `export_reports` permission on the workspace.                                       │
│ 2. Check recent incidents related to the `exp...                                                                │
╰───────────────────────────────────── docs/runbooks/export_to_csv_errors.md ─────────────────────────────────────╯

╭───────────────────────────────────────────── Logs Inspection Guide ─────────────────────────────────────────────╮
│ Score: 5                                                                                                        │
│                                                                                                                 │
│ ...log snippet:                                                                                                 │
│                                                                                                                 │
│ 1. Look for error codes (e.g., `ERR_EXPORT_TIMEOUT`, `ERR_AUTH_INVALID_TOKEN`).                                 │
│ 2. Match error codes to known runbooks such as **Export to CSV Errors** or                                      │
│    **Authentication Failures**.                                                                                 │
│ 3. Pay attention to correlation IDs and timestamps; these may be needed if you                                  │
│    escalate to engin...                                                                                         │
╰──────────────────────────────────── docs/runbooks/logs_inspection_guide.md ─────────────────────────────────────╯

### 2.2 Test Tool: incidents.search

In [6]:
# Search for export-related incidents
incident_results = await mcp_client.call_incidents_search(
    query="export timeout",
    max_results=3,
    status_filter=["Investigating", "Mitigating"]  # Only active incidents
)

console.print("\n[bold cyan]Incident Search Results:[/bold cyan]")
for incident in incident_results:
    status_color = "red" if incident['status'] == "Investigating" else "yellow"
    console.print(Panel(
        f"[{status_color}]Status: {incident['status']}[/{status_color}]\n\n"
        f"{incident['summary']}\n\n"
        f"[dim]Matched tags: {', '.join(incident['matched_tags'])}[/dim]",
        title=f"[bold]{incident['incident_id']}: {incident['title']}[/bold]",
        border_style="yellow"
    ))

Incident Search Results:

╭───────────────────────────── INC-1001: Export to CSV timeouts for large workspaces ─────────────────────────────╮
│ Status: Mitigating                                                                                              │
│                                                                                                                 │
│ Some customers with very large workspaces are experiencing timeouts when exporting dashboards to CSV. The       │
│ export_service is partially degraded while we roll out a fix.                                                   │
│                                                                                                                 │
│ Matched tags: export, timeout                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### 2.3 Test Tool: status.check

In [7]:
# Check multiple services
services = ["export_service", "auth_service", "workspace_service"]

console.print("\n[bold cyan]Service Status:[/bold cyan]")
table = Table(title="Current Service Health")
table.add_column("Service", style="cyan")
table.add_column("Status", style="magenta")
table.add_column("Notes", style="white")

for service in services:
    status = await mcp_client.call_status_check(service)
    
    status_emoji = {
        "Healthy": "✓",
        "Degraded": "⚠",
        "Outage": "✗",
        "Unknown": "?"
    }.get(status['status'], "?")
    
    table.add_row(
        status['service_name'],
        f"{status_emoji} {status['status']}",
        status['notes'][:50] + "..."
    )

console.print(table)

Service Status:

                                  Current Service Health                                  
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Service           ┃ Status     ┃ Notes                                                 ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ export_service    │ ⚠ Degraded │ Exports may time out for very large workspaces whi... │
│ auth_service      │ ✓ Healthy  │ No known issues. Recent SSO problems appear tied t... │
│ workspace_service │ ✓ Healthy  │ No active incidents. A previous permissions cachin... │
└───────────────────┴────────────┴───────────────────────────────────────────────────────┘

## 3. Process a Complete Ticket

Let's run **TICKET-001** (Export timeout) through the entire multi-agent pipeline.

In [8]:
# Load TICKET-001
ticket_data = sample_tickets[0]  # First sample
ticket = Ticket(**ticket_data)

console.print("\n[bold magenta]Processing Ticket[/bold magenta]")
console.print(Panel(
    f"[yellow]ID:[/yellow] {ticket.id}\n\n"
    f"[yellow]Description:[/yellow] {ticket.description}\n\n"
    f"[yellow]Log Snippet:[/yellow]\n{ticket.log_snippet}",
    title="[bold]Ticket Details[/bold]",
    border_style="blue"
))

Processing Ticket

╭──────────────────────────────────────────────── Ticket Details ─────────────────────────────────────────────────╮
│ ID: TICKET-001                                                                                                  │
│                                                                                                                 │
│ Description: Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout'    │
│ after ~30 seconds.                                                                                              │
│                                                                                                                 │
│ Log Snippet:                                                                                                    │
│ 2025-11-01T09:21:10Z ERR_EXPORT_TIMEOUT workspace_id=ws_1234 service=export_service                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
# Run the complete flow (this may take 10-20 seconds)
console.print("\n[bold cyan]Running multi-agent flow...[/bold cyan]")
console.print("[dim](This will take 10-20 seconds - 4 LLM calls + 3 tool calls)[/dim]\n")

supervisor_output, trace = await run_ticket_flow(ticket)

console.print("[bold green]✓ Flow complete![/bold green]\n")

Running multi-agent flow...

(This will take 10-20 seconds - 4 LLM calls + 3 tool calls)

✓ Flow complete!

### 3.1 View Final Output

In [10]:
# Display customer reply
console.print(Panel(
    supervisor_output.final_customer_reply,
    title="[bold green]Customer Reply[/bold green]",
    border_style="green",
    padding=(1, 2)
))

╭──────────────────────────────────────────────── Customer Reply ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  I understand how frustrating it can be when an export doesn't work as expected. It appears that the system is  │
│  timing out while trying to export your workspace dashboard to CSV after about 30 seconds. If possible, you     │
│  might want to try waiting a little longer or breaking down your export into smaller parts, as this can         │
│  sometimes help. Additionally, I recommend checking our documentation on 'Export to CSV Errors' for further     │
│  insights. If the issue continues, please let us know, and we can investigate further. Thank you for your       │
│  patience.                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
# Display internal note
console.print(Panel(
    supervisor_output.final_internal_note,
    title="[bold yellow]Internal Note[/bold yellow]",
    border_style="yellow",
    padding=(1, 2)
))

╭───────────────────────────────────────────────── Internal Note ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Customer is experiencing a timeout error when exporting their workspace dashboard to CSV, receiving the error  │
│  'Export failed: timeout' after ~30 seconds. This aligns with current incident INC-1001 regarding export        │
│  timeouts for large workspaces, which is still in a mitigating state. Suggested next steps include advising     │
│  the customer to attempt smaller exports and directing them to relevant documentation on export errors.         │
│  Monitor the situation and escalate if the issue persists without resolution. Runbooks applicable: 'Export to   │
│  CSV Errors', 'Workspace Permissions Issues'. No immediate action required unless the issue escalates.          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
# Display supervisor review notes
if not supervisor_output.approved:
    console.print(Panel(
        supervisor_output.review_notes,
        title="[bold red]Supervisor Review[/bold red]",
        border_style="red"
    ))
else:
    console.print(f"[green]✓ Supervisor approved without changes[/green]")
    console.print(f"[dim]Review notes: {supervisor_output.review_notes}[/dim]")

╭─────────────────────────────────────────────── Supervisor Review ───────────────────────────────────────────────╮
│ Edited customer reply for tone and clarity. Removed sarcastic remarks to maintain professionalism and ensure    │
│ the message is clear and helpful.                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 4. Explore the Trace

The trace shows **every step** the system took to process this ticket.

In [13]:
# Print trace summary
print_trace_summary(trace)

Trace Summary

Trace ID: 12685e2e-4e4f-463c-903a-bd123f8646a3

Ticket ID: TICKET-001

Total Events: 11

Agents Involved: ActionAgent, ResearchAgent, SupervisorAgent, TriageAgent

LLM Calls: 3

Tool Calls: 2

In [14]:
# Show full trace as table
render_trace_to_stdout(trace)

                         Trace: 12685e2e-4e4f-463c-903a-bd123f8646a3 (Ticket: TICKET-001)                          
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp    ┃ Agent           ┃ Step Type    ┃ Detail                                                          ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 11:49:08.472 │ TriageAgent     │ LLM_CALL     │ Analyzing ticket and creating triage plan                       │
│ 11:49:11.511 │ TriageAgent     │ AGENT_OUTPUT │ Classified as POSSIBLE_BUG, will use tools:                     │
│              │                 │              │ support_docs.search, incidents.searc...                         │
│ 11:49:11.511 │ -               │ TOOL_CALL    │ Calling tool: support_docs.search                               │
│ 11:49:11.515 │ -               │ TOOL_RESULT  │ Tool result: support_docs.search                                │
│ 11:49:11.515 │ -               │ TOOL_CALL    │ Calling tool: incidents.search                                  │
│ 11:49:11.516 │ -               │ TOOL_RESULT  │ Tool result: incidents.search                                   │
│ 11:49:11.516 │ ResearchAgent   │ AGENT_OUTPUT │ Gathered 3 docs, 3 incidents, 0 status checks                   │
│ 11:49:11.516 │ ActionAgent     │ LLM_CALL     │ Drafting customer reply and internal note                       │
│ 11:49:19.882 │ ActionAgent     │ AGENT_OUTPUT │ Drafted customer reply and internal note                        │
│ 11:49:19.882 │ SupervisorAgent │ LLM_CALL     │ Reviewing action output for quality and safety                  │
│ 11:49:29.271 │ SupervisorAgent │ AGENT_OUTPUT │ Review complete: Edited                                         │
└──────────────┴─────────────────┴──────────────┴─────────────────────────────────────────────────────────────────┘

In [15]:
# Analyze trace events by type
event_counts = {}
for event in trace.events:
    event_counts[event.step_type] = event_counts.get(event.step_type, 0) + 1

console.print("\n[bold cyan]Event Breakdown:[/bold cyan]")
for event_type, count in event_counts.items():
    console.print(f"  {event_type}: {count}")

Event Breakdown:

LLM_CALL: 3

AGENT_OUTPUT: 4

TOOL_CALL: 2

TOOL_RESULT: 2

In [16]:
# Show just LLM calls with details
console.print("\n[bold cyan]LLM Calls Detail:[/bold cyan]")
for i, event in enumerate([e for e in trace.events if e.step_type == "LLM_CALL"], 1):
    console.print(f"\n[yellow]{i}. {event.agent_name}[/yellow]")
    console.print(f"   Time: {event.timestamp}")
    console.print(f"   Detail: {event.detail}")

LLM Calls Detail:

1. TriageAgent

Time: 2025-11-15T11:49:08.472840Z

Detail: Analyzing ticket and creating triage plan

2. ActionAgent

Time: 2025-11-15T11:49:11.516444Z

Detail: Drafting customer reply and internal note

3. SupervisorAgent

Time: 2025-11-15T11:49:19.882799Z

Detail: Reviewing action output for quality and safety

In [17]:
# Show just tool calls
console.print("\n[bold cyan]Tool Calls Detail:[/bold cyan]")
tool_calls = [e for e in trace.events if e.step_type == "TOOL_CALL"]
for i, event in enumerate(tool_calls, 1):
    console.print(f"\n[yellow]{i}. {event.detail}[/yellow]")
    console.print(f"   Time: {event.timestamp}")
    if event.payload:
        console.print(f"   Input: {event.payload}")

Tool Calls Detail:

1. Calling tool: support_docs.search

Time: 2025-11-15T11:49:11.511681Z

Input: {'query': "Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: 
timeout' after ~30 seconds."}

2. Calling tool: incidents.search

Time: 2025-11-15T11:49:11.515014Z

Input: {'query': "Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: 
timeout' after ~30 seconds."}

## 5. Test Individual Agents

Let's run each agent separately to understand what they do.

### 5.1 TriageAgent - Classify & Plan

In [18]:
# Create a new ticket for testing
test_ticket = Ticket(
    id="TEST-001",
    description="User cannot log in via SSO. Error says 'SAML assertion expired'.",
    log_snippet="2025-11-14T10:00:00Z ERR_AUTH_SAML service=auth_service",
    screenshot_path=None
)

# Create trace
test_trace = Trace(trace_id="test-001", ticket_id="TEST-001")

# Run triage
triage_agent = TriageAgent()
triage_plan = await triage_agent.run(test_ticket, test_trace)

console.print("\n[bold cyan]Triage Plan:[/bold cyan]")
console.print(Panel(
    f"[yellow]Issue Type:[/yellow] {triage_plan.issue_type}\n\n"
    f"[yellow]Summary:[/yellow] {triage_plan.issue_summary}\n\n"
    f"[yellow]Tools to Call:[/yellow] {', '.join(triage_plan.tools_to_call)}\n\n"
    f"[yellow]Notes:[/yellow] {triage_plan.notes}",
    title="TriageAgent Output",
    border_style="blue"
))

Triage Plan:

╭────────────────────────────────────────────── TriageAgent Output ───────────────────────────────────────────────╮
│ Issue Type: KNOWN_ISSUE                                                                                         │
│                                                                                                                 │
│ Summary: User cannot log in via SSO due to 'SAML assertion expired' error.                                      │
│                                                                                                                 │
│ Tools to Call: support_docs.search, incidents.search                                                            │
│                                                                                                                 │
│ Notes: This seems to be a known issue related to SAML authentication that may have documented solutions.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### 5.2 ResearchAgent - Gather Context

In [19]:
# Run research (uses the plan from triage)
research_agent = ResearchAgent()
research_report = await research_agent.run(
    test_ticket,
    triage_plan,
    mcp_client,
    test_trace
)

console.print("\n[bold cyan]Research Report:[/bold cyan]")
console.print(Panel(
    research_report.summary,
    title="Research Summary",
    border_style="green"
))

if research_report.docs_results:
    console.print(f"\n[cyan]Found {len(research_report.docs_results)} documentation entries[/cyan]")
    for doc in research_report.docs_results:
        console.print(f"  • {doc['title']}")

if research_report.incident_results:
    console.print(f"\n[cyan]Found {len(research_report.incident_results)} incidents[/cyan]")
    for inc in research_report.incident_results:
        console.print(f"  • {inc['incident_id']}: {inc['title']} ({inc['status']})")

if research_report.status_results:
    console.print(f"\n[cyan]Checked {len(research_report.status_results)} services[/cyan]")
    for svc in research_report.status_results:
        console.print(f"  • {svc['service_name']}: {svc['status']}")

Research Report:

╭─────────────────────────────────────────────── Research Summary ────────────────────────────────────────────────╮
│ Found 3 relevant documentation entries:                                                                         │
│   - Authentication Failures                                                                                     │
│   - Workspace Permissions Issues                                                                                │
│                                                                                                                 │
│ Found 3 related incidents:                                                                                      │
│   - INC-1002: Authentication failures for SSO users in EU region (Investigating)                                │
│   - INC-1001: Export to CSV timeouts for large workspaces (Mitigating)                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Found 3 documentation entries

• Authentication Failures

• Workspace Permissions Issues

• Logs Inspection Guide

Found 3 incidents

• INC-1002: Authentication failures for SSO users in EU region (Investigating)

• INC-1001: Export to CSV timeouts for large workspaces (Mitigating)

• INC-1003: Workspace permissions caching bug (Resolved)

### 5.3 ActionAgent - Draft Responses

In [20]:
# Run action agent
action_agent = ActionAgent()
action_output = await action_agent.run(
    test_ticket,
    triage_plan,
    research_report,
    test_trace
)

console.print("\n[bold cyan]Action Agent Drafts:[/bold cyan]")
console.print(Panel(
    action_output.customer_reply,
    title="[green]Customer Reply (Draft)[/green]",
    border_style="green"
))
console.print(Panel(
    action_output.internal_note,
    title="[yellow]Internal Note (Draft)[/yellow]",
    border_style="yellow"
))

Action Agent Drafts:

╭──────────────────────────────────────────── Customer Reply (Draft) ─────────────────────────────────────────────╮
│ Oh dear, it seems like you're having a little trouble logging in. The error message you're seeing, 'SAML        │
│ assertion expired', is just a fancy way of saying that the login token has decided to take an early vacation.   │
│ Isn't that delightful? This is a known issue, so you're definitely not alone in this little adventure.          │
│                                                                                                                 │
│ To get back to enjoying our services, I recommend waiting a few moments and then trying to log in again. If     │
│ you're feeling extra ambitious, you might want to check out our documentation on [Authentication                │
│ Failures](docs/runbooks/authentication_failures.md) for some additional tips.                                   │
│                                                                                                                 │
│ In the meantime, we're aware of this situation and are working on it, so no need to panic. Just sit tight and   │
│ we’ll get through this together – eventually.                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Internal Note (Draft) ─────────────────────────────────────────────╮
│ User reported SSO login failure with 'SAML assertion expired' error. This is a known issue associated with      │
│ INC-1002, which is currently under investigation. Recommend referencing the 'Authentication Failures' runbook   │
│ for user guidance. No immediate action required from the user; monitor the incident for updates. If the issue   │
│ persists beyond expected resolution time, consider escalating to the support manager for further action.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### 5.4 SupervisorAgent - Review & Approve

In [21]:
# Run supervisor
supervisor_agent = SupervisorAgent()
supervisor_result = await supervisor_agent.run(
    test_ticket,
    triage_plan,
    research_report,
    action_output,
    test_trace
)

console.print("\n[bold cyan]Supervisor Review:[/bold cyan]")
approval_color = "green" if supervisor_result.approved else "red"
approval_text = "APPROVED" if supervisor_result.approved else "NEEDS CHANGES"

console.print(Panel(
    f"[{approval_color}]Status: {approval_text}[/{approval_color}]\n\n"
    f"[yellow]Review Notes:[/yellow]\n{supervisor_result.review_notes}",
    title="Supervisor Decision",
    border_style=approval_color
))

if not supervisor_result.approved:
    console.print("\n[bold]Changes Made:[/bold]")
    console.print(Panel(
        supervisor_result.final_customer_reply,
        title="Final Customer Reply (Edited)",
        border_style="green"
    ))

Supervisor Review:

╭────────────────────────────────────────────── Supervisor Decision ──────────────────────────────────────────────╮
│ Status: NEEDS CHANGES                                                                                           │
│                                                                                                                 │
│ Review Notes:                                                                                                   │
│ Revised customer reply to remove informal language and ensure a professional tone. Removed phrases like 'little │
│ trouble' and 'fancy way of saying' to maintain clarity. Adjusted the tone to be more supportive and concise     │
│ while retaining all necessary information.                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Changes Made:

╭───────────────────────────────────────── Final Customer Reply (Edited) ─────────────────────────────────────────╮
│ We understand you're having trouble logging in. The error message you're seeing, 'SAML assertion expired',      │
│ indicates that the login token has expired. This is a known issue, and you're not alone in this situation.      │
│                                                                                                                 │
│ To resolve this, please wait a few moments and then try logging in again. For additional tips, you can check    │
│ out our documentation on [Authentication Failures](docs/runbooks/authentication_failures.md).                   │
│                                                                                                                 │
│ Rest assured, we are aware of this situation and are actively working on it. Thank you for your patience.       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 6. Test Different Ticket Types

Let's run all 4 sample tickets to see different behaviors.

In [22]:
# Process all sample tickets
results = []

for ticket_data in sample_tickets:
    console.print(f"\n[bold magenta]Processing {ticket_data['id']}...[/bold magenta]")
    
    ticket = Ticket(**ticket_data)
    supervisor, trace = await run_ticket_flow(ticket)
    
    results.append({
        "ticket_id": ticket.id,
        "description": ticket.description[:50] + "...",
        "events": len(trace.events),
        "llm_calls": len([e for e in trace.events if e.step_type == "LLM_CALL"]),
        "tool_calls": len([e for e in trace.events if e.step_type == "TOOL_CALL"]),
        "approved": supervisor.approved
    })
    
    console.print(f"[green]✓ Complete[/green]")

# Show summary table
console.print("\n[bold cyan]All Tickets Summary:[/bold cyan]")
summary_table = Table(title="Processing Results")
summary_table.add_column("Ticket", style="cyan")
summary_table.add_column("Description", style="white")
summary_table.add_column("Events", justify="right")
summary_table.add_column("LLM Calls", justify="right")
summary_table.add_column("Tool Calls", justify="right")
summary_table.add_column("Approved", justify="center")

for result in results:
    summary_table.add_row(
        result["ticket_id"],
        result["description"],
        str(result["events"]),
        str(result["llm_calls"]),
        str(result["tool_calls"]),
        "✓" if result["approved"] else "✗"
    )

console.print(summary_table)

Processing TICKET-001...

✓ Complete

Processing TICKET-002...

✓ Complete

Processing TICKET-003...

✓ Complete

Processing TICKET-004...

✓ Complete

All Tickets Summary:

                                                Processing Results                                                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Ticket     ┃ Description                                           ┃ Events ┃ LLM Calls ┃ Tool Calls ┃ Approved ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ TICKET-001 │ Customer cannot export their main workspace dashbo... │     11 │         3 │          2 │    ✗     │
│ TICKET-002 │ Several SSO users in the EU are failing to log in ... │     13 │         3 │          3 │    ✗     │
│ TICKET-003 │ User reports they are an editor on a workspace but... │      9 │         3 │          1 │    ✗     │
│ TICKET-004 │ Customer's export API integration started failing ... │     11 │         3 │          2 │    ✗     │
└────────────┴───────────────────────────────────────────────────────┴────────┴───────────┴────────────┴──────────┘

## 7. Cleanup

In [23]:
# Disconnect from MCP server
await mcp_client.disconnect()
console.print("[green]✓ Disconnected from MCP server[/green]")

RuntimeError: Attempted to exit cancel scope in a different task than it was entered in

---

## Summary

In this notebook, we:

1. ✅ Connected to the MCP server and tested all 3 tools
2. ✅ Processed a complete ticket through the multi-agent pipeline
3. ✅ Explored observability traces in detail
4. ✅ Tested each agent individually to understand their roles
5. ✅ Processed all sample tickets and compared results

## Key Takeaways

- **MCP provides standardized tool integration** - No custom protocols needed
- **Multi-agent architecture enables modularity** - Each agent has a clear responsibility
- **Observability is critical** - Traces show exactly what happened
- **Each ticket type behaves differently** - The system adapts based on triage

## Next Steps

Try modifying:
- Agent prompts (in `support_copilot_host/agents.py`)
- Tool search algorithms (in `support_mcp_server/tools_*.py`)
- Add new sample tickets and see how the system handles them

---

**Happy experimenting!** 🚀